# Development Scratch

This notebook contains ideas/scratch/notes for development

In [1]:
import os
import sys
from distutils.util import strtobool

PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, PWD)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
django.setup()

In [2]:
import datetime
import django
import pandas as pd

from availabilities.models import *
from availabilities.utils import check_availabilities
from availabilities.utils import get_availabilities_for_participants
from events.models import *
from events.utils import get_all_event_participants
from collections import defaultdict

# os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
# django.setup()

In [3]:
FEB_3_9AM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=9,
    tzinfo=datetime.timezone.utc
)

FEB_3_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

FEB_3_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_3_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=3, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

FEB_4_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=4, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_4_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=4, 
    hour=17,
    tzinfo=datetime.timezone.utc
)


FEB_5_2PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=5, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_5_5PM = datetime.datetime(
    year=2022, 
    month=2, 
    day=5, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

In [4]:
# Generates test data: Not available 2022/2/3 9-5PM
andrew_defaults = {
    "date_time_start": FEB_3_9AM,
    "date_time_end": FEB_3_5PM,
    "is_available": False
}

jessie_defaults = {
    "date_time_start": FEB_4_2PM,
    "date_time_end": FEB_4_5PM,
    "is_available": False
}

andrew, __ = User.objects.get_or_create(username="Andrew")
jessie, __ = User.objects.get_or_create(username="Jessie")

availability, __ = DateAvailability.objects.update_or_create(
    user=andrew,
    category="hockey_team",
    defaults=andrew_defaults
)

availability, __ = DateAvailability.objects.update_or_create(
    user=jessie,
    category="social_stuff",
    defaults=jessie_defaults
)

# Generates event
event_defaults = {
    "members": []
}
event, __ = Event.objects.get_or_create(
    name="Hockey Party",
)
event.save()

# Connects users to event
UserEvent.objects.update_or_create(
    user=andrew,
    event=event,
    defaults={
        "is_host": True,
        "is_required": True
    }
)
UserEvent.objects.update_or_create(
    user=jessie,
    event=event,
    defaults={
        "is_host": False,
        "is_required": True
    }
)

# Generates proposed times
proposed_times = [
    (FEB_3_2PM, FEB_3_5PM),
    (FEB_4_2PM, FEB_4_5PM),
    (FEB_5_2PM, FEB_5_5PM),
]
for proposed_start, proposed_end in proposed_times:
    EventTime.objects.update_or_create(
        event=event,
        date_time_start=proposed_start,
        defaults={"date_time_end": proposed_end},
    )

# Peeks
event_time_df = pd.DataFrame(EventTime.objects.filter(event=event).values())
event_time_df

,id,event_id,date_time_start,date_time_end
0,11,1,2022-02-03 14:00:00+00:00,2022-02-03 17:00:00+00:00
1,12,1,2022-02-04 14:00:00+00:00,2022-02-04 17:00:00+00:00
2,13,1,2022-02-05 14:00:00+00:00,2022-02-05 17:00:00+00:00


In [5]:
from events.finite_state_machine import UserEventMachine

# Reset
uets = UserEventTime.objects.filter(
    user=jessie,
    event_time__event=event
)
for uet in uets:
    uet.is_active=False
    uet.has_seen=False
    uet.explicit_response=UserEventTime.NO_RESPONSE
    uet.save()
user_event = UserEvent.objects.get(user=jessie, event=event)
user_event.state = UserEvent.NO_COMMUNICATION
user_event.save()

user_event = UserEvent.objects.get(user=jessie, event=event)
user_event_machine = UserEventMachine(user_event)
user_event_machine.send_initial_text()
user_event_machine.process_input_text("No")

Would 2022-02-05 14:00:00+00:00 to 2022-02-05 17:00:00+00:00 work for you?
New State: waiting_response
There are no viable event times, can you please suggest a time that works for you?
New State: waiting_suggestion


In [6]:
pd.DataFrame(UserEvent.objects.values())

,id,state,user_id,event_id,is_required,is_host
0,4,None,1,1,True,True
1,5,waiting_suggestion,2,1,True,False


In [8]:
# def process_suggested_date(user, event, suggested_event_time):
#     """
#     Compares suggested_event_time against all members in event
#     """
#     participants = get_all_event_participants(event)
#     availability_dict, times_that_work = check_availabilities(event)
    
#     time_works = suggested_event_time.id in times_that_work
#     available_participants = availability_dict[suggested_event_time.id]
#     unavailable_participants = list(set(participants) - set(available_participants))
        
#     return time_works, unavailable_participants
    

# user = andrew
# suggested_event_time = EventTime.objects.get(
#     event_id=event.id,
#     date_time_start=FEB_5_2PM
# )

# process_suggested_date(user, event, suggested_event_time)